In [1]:
from platform import python_version
print(python_version())

3.7.2


In [1]:
from pandas.io.json import json_normalize
from bson import json_util, ObjectId
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import json
import re

In [2]:
mongo = pymongo.MongoClient("mongodb://localhost:27017/")
database = mongo["DB_CRAN"]
collection = database["R_packages"]

In [3]:
collection_packages = list(collection.find({}, {'_id': False}))

packages = []
for c in collection_packages:
    packages.append(json.loads(json_util.dumps(c)))
      
print(f'HAY {len(packages)} DOCUMENTOS')

HAY 15368 DOCUMENTOS


In [4]:
list_pkgs = []
for i in packages:
    for k, v in i.items():
        list_pkgs.append(k)

versions = []
con = 0

for i in list_pkgs:
    if (con <= len(packages)):
        versions.append(packages[con][i]['versions'])
        con = con + 1

versions_pkgs = []

for i in versions:
    for k, v in i.items():
        versions_pkgs.append(v)
        
results = []

for i in versions_pkgs:
    pkg = {}
    for k, v in i.items():
        if isinstance(v, dict):
            value = [l for l in v ]
            version = [''.join(l) for l in v.values() ]
            
            pkg[k] = value
            
            for version_R, version in zip(value, version):
                if version_R == "R":
                    pkg.update({ "Version_" + version_R :''.join(version) })
                    
        elif isinstance(v, list):
            if len(v) <= 1 and k != "releases":
                pkg[k] = ''.join(v)
            else:
                pkg[k] = v
    results.append(json.loads(json_util.dumps(pkg)))

df = pd.DataFrame(json_normalize(results))

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [5]:
versions_pkgs[0]

{'Package': ['A3'],
 'Type': ['Package'],
 'Title': ['A3: Accurate, Adaptable, and Accessible Error Metrics for<U+000a>Predictive Models'],
 'Version': ['0.9.1'],
 'Date': ['2013-02-06'],
 'Author': ['Scott Fortmann-Roe'],
 'Maintainer': ['Scott Fortmann-Roe <scottfr@berkeley.edu>'],
 'Description': ['This package supplies tools for tabulating and analyzing<U+000a>the results of predictive models. The methods employed are<U+000a>applicable to virtually any predictive model and make<U+000a>comparisons between different methodologies straightforward.'],
 'License': ['GPL (>= 2)'],
 'Depends': {'xtable': ['*'], 'pbapply': ['*']},
 'Suggests': {'randomForest': ['*'], 'e1071': ['*']},
 'Packaged': ['2013-02-06 16:46:12 UTC; scott'],
 'Repository': ['CRAN'],
 'Date/Publication': ['2013-02-07 10:00:27'],
 'crandb_file_date': ['2013-02-07 04:00:29'],
 'date': ['2013-02-07T10:00:27+00:00'],
 'releases': ['2.15.3']}

In [24]:
results[143]

{'Package': 'acc',
 'Type': 'Package',
 'Title': 'Exploring Accelerometer Data',
 'Version': '1.3.3',
 'Date': '2016-12-15',
 'Author': 'Jaejoon Song, Matthew G. Cox',
 'Maintainer': 'Jaejoon Song <jjsong2@mdanderson.org>',
 'Description': 'Processes accelerometer data from uni-axial and tri-axial devices,\nand generates data summaries. Also includes functions to plot, analyze, and\nsimulate accelerometer data.',
 'Depends': ['R', 'mhsmm'],
 'Version_R': '>= 2.10',
 'Imports': ['zoo',
  'PhysicalActivity',
  'nleqslv',
  'plyr',
  'methods',
  'DBI',
  'RSQLite',
  'circlize',
  'ggplot2',
  'R_utils',
  'iterators',
  'Rcpp'],
 'License': 'GPL (>= 2)',
 'LinkingTo': ['Rcpp', 'RcppArmadillo'],
 'RoxygenNote': '5.0.1',
 'NeedsCompilation': 'yes',
 'Packaged': '2016-12-16 22:14:21 UTC; JS',
 'Repository': 'CRAN',
 'Date/Publication': '2016-12-16 23:29:38',
 'crandb_file_date': '2016-12-16 22:32:41',
 'date': '2016-12-16T22:29:38+00:00',
 'releases': []}

In [25]:
records = []

for item in results:
    records.append((item['Package'],item['Version'],item['date'],item['Depends']))

KeyError: 'Depends'

In [26]:
df_original = pd.DataFrame(json_normalize(results))
df_original.head(10)

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,Package,Type,Title,Version,Date,Author,Maintainer,Description,License,Depends,...,Config/reticulate,Co-author,Saveimage,SystemRequirement,Versionnote,Classification_ACM,ReleaseDate,xxxLazyLoad,Github URL,Github
0,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.1,2013-02-06,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.2,2013-03-24,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,Package,"Accurate, Adaptable, and Accessible Error Metr...",1.0.0,2015-08-15,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,Supplies tools for tabulating and analyzing th...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aaSEA,Package,Amino Acid Substitution Effect Analyser,1.0.0,NaN,Raja Sekhara Reddy D.M,Raja Sekhara Reddy D.M <raja.duvvuru@gmail.com>,"Given a protein multiple sequence alignment, i...",GPL-3,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aaSEA,Package,Amino Acid Substitution Effect Analyser,1.1.0,NaN,Raja Sekhara Reddy D.M,Raja Sekhara Reddy D.M <raja.duvvuru@gmail.com>,"Given a protein multiple sequence alignment, i...",GPL-3,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ABACUS,NaN,Apps Based Activities for Communicating and Un...,1.0.0,NaN,"Mintu Nath [aut, cre]",Mintu Nath <dr.m.nath@gmail.com>,A set of Shiny apps for effective communicatio...,GPL-3,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,abbyyR,NaN,Access to Abbyy Optical Character Recognition ...,0.1,NaN,"Gaurav Sood [aut, cre]",Gaurav Sood <gsood07@gmail.com>,Get text from images of text using Abbyy Cloud...,GPL (>= 2),[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,abbyyR,NaN,Access to Abbyy Optical Character Recognition ...,0.2,NaN,"Gaurav Sood [aut, cre]",Gaurav Sood <gsood07@gmail.com>,Get text from images of text using Abbyy Cloud...,GPL (>= 2),[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,abbyyR,NaN,Access to Abbyy Optical Character Recognition ...,0.2.1,NaN,"Gaurav Sood [aut, cre]",Gaurav Sood <gsood07@gmail.com>,Get text from images of text using Abbyy Cloud...,MIT + file LICENSE,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,abbyyR,NaN,Access to Abbyy Optical Character Recognition ...,0.2.2,NaN,"Gaurav Sood [aut, cre]",Gaurav Sood <gsood07@gmail.com>,Get text from images of text using Abbyy Cloud...,MIT + file LICENSE,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_original.shape

(91321, 224)

In [28]:
df_original.isna().sum().to_frame().head(120)

,0
Package,0
Type,41561
Title,150
Version,19
Date,23216
...,...
Contributor,91319
Respository,91318
lastDate,91320
LicenseNote,91288


In [29]:
df = df_original.dropna(axis=1, thresh=10000)
df.shape

(91321, 30)

In [30]:
print(df.columns.tolist())

['Package', 'Type', 'Title', 'Version', 'Date', 'Author', 'Maintainer', 'Description', 'License', 'Depends', 'Suggests', 'Packaged', 'Repository', 'Date/Publication', 'crandb_file_date', 'date', 'releases', 'Version_R', 'NeedsCompilation', 'Imports', 'Encoding', 'LazyData', 'RoxygenNote', 'VignetteBuilder', 'MD5sum', 'URL', 'Authors@R', 'BugReports', 'LinkingTo', 'LazyLoad']


In [37]:
columns = ['Author','Imports','Depends','Suggests','LinkingTo']

for col in columns:
    df.loc[df[col].isnull(),[col]] = df.loc[df[col].isnull(), col].apply(lambda x: [])

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [38]:
df.loc[df['Date/Publication'].isna(),'Date/Publication'] = df['date']

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df['DatePublication'] = pd.to_datetime(df['Date/Publication'], utc=True).values.astype('M8[D]')
df['Year/Publication'] = df['DatePublication'].dt.strftime('%Y')
df['Month/Publication'] = df['DatePublication'].dt.strftime('%m')
df['Day/Publication'] = df['DatePublication'].dt.strftime('%d')
df["Year_Month/Publication"] = df['DatePublication'].dt.strftime('%Y-%m')

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [40]:
df.head()

,Author,Authors@R,BugReports,Date,Date/Publication,Depends,Description,Encoding,Imports,LazyData,...,Version_R,VignetteBuilder,crandb_file_date,date,releases,DatePublication,Year/Publication,Month/Publication,Day/Publication,Year_Month/Publication
0,Scott Fortmann-Roe,NaN,NaN,2013-02-06,2013-02-07 10:00:27,"[xtable, pbapply]",This package supplies tools for tabulating and...,NaN,[],NaN,...,NaN,NaN,2013-02-07 04:00:29,2013-02-07T10:00:27+00:00,[2.15.3],2013-02-07,2013,02,07,2013-02
1,Scott Fortmann-Roe,NaN,NaN,2013-03-24,2013-03-26 19:58:40,"[R, xtable, pbapply]",This package supplies tools for tabulating and...,NaN,[],NaN,...,>= 2.15.0,NaN,2013-03-26 14:58:40,2013-03-26T19:58:40+00:00,"[3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.0, 3.1.1]",2013-03-26,2013,03,26,2013-03
2,Scott Fortmann-Roe,NaN,NaN,2015-08-15,2015-08-16 23:05:52,"[R, xtable, pbapply]",Supplies tools for tabulating and analyzing th...,NaN,[],NaN,...,>= 2.15.0,NaN,2015-08-16 17:08:22,2015-08-16T23:05:52+00:00,[],2015-08-16,2015,08,16,2015-08
3,Raja Sekhara Reddy D.M,NaN,NaN,NaN,2019-08-01 09:10:06 UTC,[R],"Given a protein multiple sequence alignment, i...",UTF-8,"[DT, networkD3, shiny, shinydashboard, magritt...",true,...,>= 3.4.0,knitr,2019-08-01 09:50:23,2019-08-01T08:10:06+00:00,[],2019-08-01,2019,08,01,2019-08
4,Raja Sekhara Reddy D.M,NaN,NaN,NaN,2019-11-09 16:20:02 UTC,[R],"Given a protein multiple sequence alignment, i...",UTF-8,"[DT, networkD3, shiny, shinydashboard, magritt...",true,...,>= 3.4.0,knitr,2019-11-09 17:56:27,2019-11-09T15:20:02+00:00,[],2019-11-09,2019,11,09,2019-11


In [41]:
df['NumberOfImports'] = df['Imports'].str.len()
df['NumberOfDepends'] = df['Depends'].str.len()
df['NumberOfSuggests'] = df['Suggests'].str.len()
df['NumberOfLinkingTo'] = df['LinkingTo'].str.len()

/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mmora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [42]:
colnames = df.columns.tolist()

In [43]:
df.to_csv("DATA_CRAN/CRAN_DATA_20191231.csv", header=colnames, sep="|", index=False)